<a href="https://colab.research.google.com/github/lararst/ADNIanalysis/blob/main/Causality_LCD_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions
- Make a copy of this notebook for your solutions
- For submission, click "Share" and allow anyone with link to comment, so that I can easily give feedback. Then submit the sharing link to the ELO.
- Only change code between `### BEGIN SOLUTION` and `### END SOLUTION`
- Do not include any additional code cells in your submission.
- Your solution should be on Google Colab.
- When doing "Restart and Run all", all code should run without errors, so all tests must pass. I will do this when grading, so if any errors occur, I cannot grade your work.



In [ ]:
# Install and load library, takes a couple of minutes.
system('sudo apt-get install -y g++ gcc gfortran-8')
install.packages("BiocManager", lib="/usr/local/lib/R/site-library", quiet=TRUE)
BiocManager::install("pcalg", lib="/usr/local/lib/R/site-library", quiet=TRUE)
install.packages("testthat", lib="/usr/local/lib/R/site-library", quiet=TRUE)
library(pcalg)
library(testthat)
library(ggplot2)
library(openssl)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.17), R 4.2.0 (2022-04-22)

Installing package(s) 'BiocVersion', 'pcalg'

also installing the dependencies ‘zoo’, ‘BiocGenerics’, ‘DEoptimR’, ‘lmtest’, ‘abind’, ‘graph’, ‘RBGL’, ‘igraph’, ‘ggm’, ‘corpcor’, ‘robustbase’, ‘vcd’, ‘Rcpp’, ‘bdsmatrix’, ‘sfsmisc’, ‘fastICA’, ‘clue’, ‘RcppArmadillo’, ‘BH’


Linking to: OpenSSL 1.1.1  11 Sep 2018



The following is a helper function to sample Bernoulli variables.

In [ ]:
bernSample <- function (p, n=1) sample(x = c(TRUE, FALSE), size = n, replace = TRUE, prob=c(p, 1-p))

In [ ]:
bernSample(0.9)
bernSample(0.5, n=10)

[1] TRUE

[1]  TRUE FALSE FALSE  TRUE  TRUE FALSE FALSE  TRUE FALSE  TRUE

# Introduction to R Data Frames
A convenient way of organising a data set in R is via data frames.

Some basic facts about R data frames:
* Given vectors `colA` and `colB`, you can create a data frame as `data.frame(a=colA, b=colB)` containing columns `a` and `b`.
* If `df` is a data frame having column `col`, then `df$col` gives the columns as a vector.
* If you have a string variable `colName <- "col"`, you can access the column named `col` in data frame `df` via `df[[colName]]`.
* You can add new columns using `df$col <- ...`

In the following example, we create a data set of samples from the Bayesian Network:
```
P(A=TRUE)=0.5
P(B=TRUE)=0.1
P(C=TRUE|A=TRUE,  B=TRUE)  = 0.9
P(C=TRUE|A=TRUE,  B=FALSE) = 0.5
P(C=TRUE|A=FALSE, B=TRUE)  = 0.4
P(C=TRUE|A=FALSE, B=FALSE) = 0.1
```
The following code constructs a function that samples a data set via ancestral sampling.

In [ ]:
sampleDemoDataset <- function(numSamples) {
  # First create A and B variables
  dfDemo <- data.frame(
    a = bernSample(.5, numSamples),
    b = bernSample(.1, numSamples)
  )

  # Now create the C column
  # We apply the function, which returns a boolean, to each row
  # This is aggregated into a new column.
  dfDemo$c <- apply(dfDemo, 1, function(r) {
    if (r['a'] && r['b']) {
      p <- 0.9
    } else if (r['a'] && !r['b']) {
      p <- 0.5
    } else if (!r['a'] && r['b']) {
      p <- 0.4
    } else if (!r['a'] && !r['b']) {
      p <- 0.1
    }
    bernSample(p)
  })
  dfDemo
}
dfDemo <- sampleDemoDataset(10)
dfDemo

a,b,c
<lgl>,<lgl>,<lgl>
TRUE,FALSE,TRUE
TRUE,FALSE,FALSE
TRUE,FALSE,FALSE
FALSE,FALSE,FALSE
FALSE,FALSE,FALSE
FALSE,FALSE,TRUE
FALSE,FALSE,FALSE
TRUE,FALSE,TRUE
TRUE,TRUE,TRUE


# Exercise 1: Sample Asia Bayesian Network
(3 points)

You can find the network here: [Asia BN](https://www.bayesserver.com/examples/networks/asia).

You should create a data frame with the following boolean columns:
* visitToAsia
* smoker
* hasTuberculosis
* hasLungCancer
* hasBronchitis
* tuberculosisOrCancer
* xRayAbnormal (note: this differs from the web page in order to make it clear what TRUE means)
* dyspnea

Below, you should create the body of the function `sampleDataset`, which samples the dataset according to the BayesNet for a given number of samples. Note that in R, the last expression in a function body is automatically returned.


In [ ]:
sampleDataset <- function(numSamples) {
### BEGIN SOLUTION
### END SOLUTION
}

df <- sampleDataset(100000)
df

In [ ]:
# Test all columns are present
expect_equal(sort(names(df)), c(
  'dyspnea', 'hasBronchitis', 'hasLungCancer', 'hasTuberculosis',
  'smoker', 'tuberculosisOrCancer', 'visitToAsia', 'xRayAbnormal'))
# Test number of rows
expect_equal(nrow(df), 100000)

# Exercise 2: G-Test
(3 points)

The next exercise is to implement the G test. Please consult the lecture notes. First, we compute the G statistic and then we use the fact that given the null hypothesis of (conditional) independence, the G statistic is drawn from the chi-squared distribution whose degrees of freedom are given by the number of values the variables take (2 for booleans) according to the formulae in the references.

You should construct four functions:
- `gStatistic(x, y)` which takes boolean vectors as arguments and returns the G statistic about the independence of `x` and `y` conditioning on the empty set.
- `gTest(x, y)` which gives the probability that at least `gStatistic(x, y)` is observed, assuming `x` and `y` are sampled from an independent distribution.
- `gStatisticCond(x, y, z)` which takes boolean vectors as arguments and returns the G statistic about the independence of `x` and `y` conditioning on variable `z`.
- `gTestCond(x, y, z)` which gives the probability that at least `gStatisticCond(x, y, z)` is observed, assuming `x` and `y` are sampled from an independent distribution given `z`.

Your implementation of the G statistics should use simple R and not rely on any external libraries, including `PCALG`, which we imported above. Your implementation of the G tests should use your G statistic functions and the function [pchisq or qchisq](https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/Chisquare).


Some hints:
-  You can make a for loop using `for (v in c(TRUE, FALSE)) { ... }`
- You can count the number of `TRUE` elements via `sum(x==TRUE)`
- You can count the number of times a row in `x` is `TRUE` and the same row in `y` is `FALSE` via `sum(x == TRUE & y == FALSE)`.
- For the function `pchisq` or `qchisq`, make sure you understand the meaning of argument `lower.tail`.
- Make sure to handle the case `n log (n)=0` for `n=0`.
- Taking the product of large numbers may overflow. Use `log(a * b) = log(a) + log(b)` to avoid this.

In [ ]:
gStatistic <- function(x, y) {
### BEGIN SOLUTION
### END SOLUTION
}

gTest <- function(x, y) {
### BEGIN SOLUTION
### END SOLUTION
}

gStatisticCond <- function(x, y, z) {
### BEGIN SOLUTION
### END SOLUTION
}
gTestCond <- function(x, y, z) {
### BEGIN SOLUTION
### END SOLUTION
}

We test our functions on the samples:

In [ ]:
# visitToAsia and smoker are d-separated, so we expect independence
# Thus we should see a high p-value (corresponding to low evidence against the null hypothesis).
gTest(df$visitToAsia, df$smoker)

# hasLungCancer and hasBronchitis are not d-separated, so we expect dependence
# Thus we should see a low p-value (corresponding to high evidence against the null hypothesis).
gTest(df$hasLungCancer, df$hasBronchitis)

# hasLungCancer and hasBronchitis are d-separated given smoker,
# so we expect independence given smoker.
# Thus we should see a high p-value (corresponding to low evidence against the null hypothesis).
gTestCond(df$hasLungCancer, df$hasBronchitis, df$smoker)

In [ ]:
# Validate empty handling
expect_equal(gTest(c(1), c(1)), 1)
expect_equal(gTestCond(c(1), c(1), c(1)), 1)

# Comparing the solution to a reference implementation
xTest = bernSample(0.3, n=100)
yTest = bernSample(0.3, n=100)
zTest = bernSample(0.3, n=100)
expect_equal(gTest(xTest, yTest), gSquareBin(x=1,y=2, S=c(), dm=cbind(xTest, yTest, zTest), verbose=FALSE))
expect_equal(gTestCond(xTest, yTest, zTest), gSquareBin(x=1,y=2, S=3, dm=cbind(xTest, yTest, zTest), verbose=FALSE))

# Exercise 3: Local Causal Discovery
(1 point)

With the (conditional) independence tests, we can see if a triple of variables `(x1, x2, x3)` satisfies the assumptions of proposition 9.7.1 of the lecture notes, if we assume that `x2` is not a cause of `x1`. A key parameter is the p-value threshold: what is the highest probability of the data under the null hypothesis of (conditional) independence such that we reject the null hypothesis. An often used value is p=0.05, meaning that if the data is sampled from the null hypothesis, we expect to falsely reject the null hypothesis 5% of the time.

In [ ]:
# Assuming x2 is not a cause of x1
lcdTest <- function(x1, x2, x3, pThreshold) {
### BEGIN SOLUTION
### END SOLUTION
}

# Exercise 4: Evaluating the LCD test
We will now evaluate whether the `lcdTest` function we've built agrees with what we expect from d-separation in the Asia Bayesian Network. We study the effect of two variables: the number of samples from the Bayesian Network and the `pThreshold` value used for the LCD test.

## 4.1 Dataset
(2 points)

Of all triples of variables for which the LCD assumption may hold, we manually construct a small dataset to evaluate our LCD testing. In the Comma Separated Value string below, replace `?` with either `TRUE` or `FALSE`, dependent on whether the LCD assumptions should hold based on d-separation in the Asia graph.

We check the correctness of the labels you provided against a hash of the correct answer.

In [ ]:
### BEGIN SOLUTION
data <- "
x1, x2, x3, label
visitToAsia, hasTuberculosis, tuberculosisOrCancer, ?
visitToAsia, smoker, tuberculosisOrCancer, ?
visitToAsia, hasLungCancer, tuberculosisOrCancer, ?
visitToAsia, dyspnea, tuberculosisOrCancer, ?
hasTuberculosis, tuberculosisOrCancer, dyspnea, ?
visitToAsia, hasTuberculosis, xRayAbnormal, ?
visitToAsia, tuberculosisOrCancer, xRayAbnormal, ?
visitToAsia, hasTuberculosis, dyspnea, ?
smoker, hasLungCancer, tuberculosisOrCancer, ?
smoker, tuberculosisOrCancer, xRayAbnormal, ?
smoker, hasLungCancer, dyspnea, ?
smoker, tuberculosisOrCancer, dyspnea, ?
xRayAbnormal, hasBronchitis, dyspnea, ?
"
### END SOLUTION

# Construct data frame from CSV
lcdEvalDataset <- read.table(
  text = data, sep =",", header = TRUE, strip.white = TRUE,
  colClasses=c("character", "character", "character", "logical"))
lcdEvalDataset

In [ ]:
# Test if the labels were correct.
# If this test fails, some of the labels are incorrect.
s <- toString(lapply(lcdEvalDataset$label, toString))
expect_equal(toString(md5(s)), "6bed508f2379c181b119d0177351e50e")

## 4.2 Run evaluation
(2 points)

Now we test the esitmation with the data set by computing the precision and recall of our LCD prediction compared to the ground truth labels set we just constructed.

Write the code to fill the `results` data frame. It should contain 25 rows, and the columns:
- numSamples: values in 1E2, 1E3, 1E4, 1E5, 1E6
- pThreshold: values in 0.001, 0.01, 0.05, 0.1, 0.2
- precision: the precision of the LCD estimation for given numSamples and pThreshold
- recall: the recall of the LCD estimation for given numSamples and pThreshold

Hint:
- You can add a row to a data frame by `results[nrow(results) + 1,] = list(numSamples, pThreshold, precision, recall)`
- At least one of the recalls and at least one of the precisions should be 1.
- Wikipedia has a definition of [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall), where relevant means that the LCD condition holds.

In [ ]:
results <- data.frame(numSamples=double(), pThreshold=double(), precision=double(), recall=double())

### BEGIN SOLUTION
### END SOLUTION
results

In [ ]:
# Should have 25 rows
expect_equal(nrow(results), 5 * 5)
# Columns should match
expect_equal(sort(names(results)), sort(c("numSamples", "pThreshold", "precision", "recall")))
# numSamples should have right values
expect_equal(sort(unique(results$numSamples)), sort(c(1E2, 1E3, 1E4, 1E5, 1E6)))
# pThreshold should have right values
expect_equal(sort(unique(results$pThreshold)), sort(c(0.001, 0.01, 0.05, 0.1, 0.2)))

# At least one setting should have perfect precision and recall
expect_true(any(results$precision == 1))
expect_true(any(results$recall == 1))


In [ ]:
# Plot results
(ggplot(data = results, aes(x = pThreshold, y = precision, group=numSamples, color=factor(numSamples)))
  + geom_line(na.rm = TRUE) + geom_point(na.rm = TRUE)
  + labs(color = "#samples", title = "Precision")
  + scale_x_log10())
  (ggplot(data = results, aes(x = pThreshold, y = recall, group=numSamples, color=factor(numSamples)))
  + geom_line(na.rm = TRUE) + geom_point(na.rm = TRUE)
  + labs(color = "#samples", title = "Recall")
  + scale_x_log10())

# Exercise 5: Infer causal effect
(1 point)

With the LCD test, we may find that for the triple:
`X1=visitToAsia`, `X2=hasTuberculosis`, `X3=xRayAbnormal`, the LCD test holds. If so, we can use `P(xRayAbnormal|do(hasTuberculosis))=P(xRayAbnormal|hasTuberculosis)`
to estimate causal effect. First, verify that this LCD condition holds. Then, to estimate the conditional probability, we can use R tables, which we then normalize. As an example, `table(df[c('visitToAsia', 'smoker')])` creates an R table from the dataframe `df` that shows the counts for all four possible values of the two boolean variables. Create such a table and normalize appropriately to infer the conditional probabilities.

Tasks:
- Test LCD test for the triple `X1=visitToAsia`, `X2=hasTuberculosis`, `X3=xRayAbnormal`
- Construct the conditional probability `P(xRayAbnormal|hasTuberculosis)`

Hints:
- Dividing a  $N \times M$ shaped table by an $N$ vector divides the elements in the $i$th row by the $i$th value of the vector.
- Consider using the function `rowSums(t)` on the table.

In [ ]:
df <- sampleDataset(1000000)

In [ ]:
### BEGIN SOLUTION
### END SOLUTION

# Exercise 6: Infer ancestor and confounder matrices

## 6.1: Construct matrices

(2 points)

If we use as prior knowledge that `visitToAsia` and `smoker` have no parents, we can use LCD to infer the *ancestor matrix*.

If a SCM has $N$ exogenous and endogenous variables, the ancestor matrix is a $N \times N$ matrix with $A_{vw}=1$ if we infer that $v$ is a ancestor of $w$.  $A_{vw}=-1$ if we infer this is not the case or $A_{vw}=0$ if the causal discovery method is inconclusive about this ancestral relationship.

Similarly, we can make a confounded matrix $C_{vw}$. If $C_{vw}=1$, we know $v$ and $w$ have a confounder, if $C_{vw}=-1$ we know they don't or if $C_{vw}=0$ we can't tell.

Hints & instructions:
- For an R matrix `m`, use `m[r, c] = v` to set the value at row `r` and column `c` to value  `v`.
- Do use the prior knowledge that `visitToAsia` and `smoker` have no parents to the matrix.
- You do not have to reason about transitivity. Just incorporate the immediate LCD results and the prior knowledge.
- The LCD algorithm is not very informative for constructing the confounded matrix $C$
- You can use `pThreshold=0.001`
- Use the 1M samples from the previous cell
- Let $A_{vv}=1, C_{vv}=-1$

In [ ]:
ancestorMatrix = matrix(data=c(0), nrow=8, ncol=8, dimnames=list(names(df), names(df)))
confoundedMatrix = matrix(data=c(0), nrow=8, ncol=8, dimnames=list(names(df), names(df)))

# The variable names other than smoker and visitToAsia
varNames = c(
  'dyspnea', 'hasBronchitis', 'hasLungCancer', 'hasTuberculosis',
  'tuberculosisOrCancer', 'xRayAbnormal')

### BEGIN SOLUTION
### END SOLUTION
ancestorMatrix
confoundedMatrix

## 6.2: Compare to FCI implementation
(1 point)

The following code computes and displays the ancestor and counfounded matrices using the [FCI](https://www.rdocumentation.org/packages/pcalg/versions/2.7-1/topics/fci) method. In the cell below, briefly comment qualitatively on the results obtained via LCD and FCI.

In [ ]:
dfFCI <- sampleDataset(100000) # Smaller dataset as FCI is slower
res <- fci(list(dm=dfFCI, adaptDF=FALSE), indepTest=binCItest, alpha = 0.01, doPdsep = TRUE, labels=names(df), jci="1", contextVars=c(1, 2) ,selectionBias=FALSE)
fciAncestorMatrix <- pag2anc(res@amat)
fciCounfounderMatrix <- pag2conf(res@amat)
fciAncestorMatrix
fciCounfounderMatrix

BEGIN SOLUTION

END SOLUTION